<font size='5'><b><u>D605 Task 3: Amazon Logistics Optimization</u></b></font>

Amazon, known primarily as an online retail company, has a large-scale logistics operation with over 50 aircrafts. They operate out of two main cargo hubs, delivering packages to their network of 68 international fulfillment centers. Three of the facilities are focus centers. They are located in ‘focus’ cities and facilitate deliveries to other fulfillment centers.

Amazon has requested an analysis of its logistics network to optimize shipping costs. Shipping expenses are one of the most significant company expenditures. Optimizing the logistics network will reduce costs and increase company profits.


<b><u><font size='3'>Objective Function</font></u></b>

The object of the optimization problem is to minimize Amazon’s cost function. Xij, Yik, and Zjk represent the decision variables. C represents the cost coefficient associated with each decision variable.


<img src="/Users/elineiman/Data Portfolio/Logistics Optimization/Images/Objective.png" alt="Alt text" width="500">

<b><u><font size='3'>Constraints </font></u></b>

Amazon’s logistics network has four main constraints:


<table>
  <tr>
    <td>
      <img src="/Users/elineiman/Data Portfolio/Logistics Optimization/Images/C1.png" alt="Constraint 1" width="300"><br>
      <b>1</b>
    </td>
    <td>
      <img src="/Users/elineiman/Data Portfolio/Logistics Optimization/Images/C2.png" alt="Constraint 2" width="300"><br>
      <b>2</b>
    </td>
  </tr>
  <tr>
    <td>
      <img src="/Users/elineiman/Data Portfolio/Logistics Optimization/Images/C3.png" alt="Constraint 3" width="300"><br>
      <b>3</b>
    </td>
    <td>
      <img src="/Users/elineiman/Data Portfolio/Logistics Optimization/Images/C4.png" alt="Constraint 4" width="300"><br>
      <b>4</b>
    </td>
  </tr>
</table>

<b>1. Main Hub Capacity:</b> The two main cargo centers have a limited size. The total quantity of goods each main hub ships out to focus cities (Xij) and fulfillment centers (Yik) can not exceed their storage capacity. The CVG hub’s capacity is 95,650 monthly tons. The AFW hub’s capacity is 44,350 monthly tons.

<b>2. Quantity Into a Focus Center:</b> Each focus center has a limited size. The total quantity each focus center receives (Xij) can not exceed its storage capacity. The quantity each focus center receives is the sum of goods from main hub 1 and main hub 2. Leipzig focus center’s capacity is 85,000 monthly tons. Hyderabad focus center’s capacity is 19,000 monthly tons. San Bernardino focus center’s capacity is 36,000 monthly tons.

<b>3. Quantity Out of a Focus Center:</b> Each focus city center receives a certain amount of goods from the main hubs (Xij). The total quantity a focus city center sends out to fulfillment centers (Zjk) is the same quantity they initially received. Therefore, excess goods are not accumulated, and received goods are not lost.

<b>4. Fulfillment Center Demand:</b> Each fulfillment center has a demand for goods based on customer needs. Fulfillment centers in cities with more customers have a higher demand for goods. The goods each fulfillment center receives from the main hubs (Yik) and focus city centers (Zjk) must meet the fulfillment center's demand.

There are 73 total constraints given two main hubs, three focus centers, and 65 fulfillment centers. There are two hub capacity constraints, three focus city quantity received constraints, three focus city quantity shipped constraints, and 65 fulfillment center demand constraints. 

<b><u><font size='3'>Decision Variables </font></u></b>

<b>Xij:</b> The quantity sent from main hub i to focus center j. 

<b>Yik:</b> The quantity sent from main hub i to fulfillment center k.

<b>Zjk:</b> The quantity sent from focus center j to fulfillment center k.

There are 189 decision variables to optimize. The logistics network has 331 potential connections, but some facilities do not interact because they are too far apart.


<b><u><font size='3'>Optimization Approach </font></u></b>

Linear programming will be used to solve Amazon’s logistics optimization problem. Linear programming involves continuous decision variables, and requires linear constraints and objective functions. The solution will utilize the simplex algorithm.

<b><u><font size ='3'>Program To Solve Optimization Problem </font></u></b>

Define Components:
- Decision Variables
- Objective Function
- Constraints

In [19]:
from pulp import LpProblem, LpVariable, LpMinimize, LpContinuous, lpSum, LpStatus
from pulp import *
import numpy as np
import pandas as pd
import math
from pulp import LpProblem, LpStatus

Create optimization scenario

In [20]:
prob = LpProblem("Amazon Logistics", LpMinimize)

/opt/anaconda3/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Import cost coefficients
- The 'costs' data frame contains the cost coefficients for shipping goods from a supplier (hub or focus) to a fulfillment center.
- The data table was extracted and imported from the task scenario description.

In [21]:
costs=pd.read_csv('/Users/elineiman/Desktop/WGU Data Sets/Cost Coefficients - Sheet1.csv')

In [22]:
#Drop first and fourth row - just meant to be headers
costs = costs.drop([0,4])
costs = costs.reset_index(drop=True)

#rename columns
costs.columns = ['Center', 'Cincinnati_hub', 'Alliance_hub', 'Leipzig_focus', 'Hyderabad_focus', 'San_Bernadino_focus']

#Align Row Names
costs.iloc[0, costs.columns.get_loc('Center')] = 'Leipzig_focus'
costs.iloc[1, costs.columns.get_loc('Center')] = 'Hyderabad_focus'
costs.iloc[2, costs.columns.get_loc('Center')] = 'San_Bernadino_focus'

In [23]:
costs.head()

,Center,Cincinnati_hub,Alliance_hub,Leipzig_focus,Hyderabad_focus,San_Bernadino_focus
0,Leipzig_focus,1.5,NaN,NaN,NaN,NaN
1,Hyderabad_focus,NaN,NaN,1.6,NaN,NaN
2,San_Bernadino_focus,0.5,0.5,NaN,NaN,NaN
3,Paris,1.6,NaN,0.5,1.1,NaN
4,Cologne,1.5,NaN,0.5,1.0,NaN


Create list of each center combinations cost coefficient
- Many centers don't interact, thus there cost coefficient is NaN.
- The list won't include combinations where the center doesn't interact.

In [24]:
cost_dict = {}

for index, row in costs.iterrows():
    center_name = row['Center']
    for col in row.index[1:]: 
        cost = row[col]
        if pd.notna(cost):
            cost_dict[(col, center_name)] = cost

Define Decision Variables:

In [25]:
hubs = ["Cincinnati_hub", "Alliance_hub"]
focus_centers = ["Leipzig_focus", "Hyderabad_focus", "San_Bernadino_focus"]
fulfillment_centers = costs['Center'][3:].tolist()

capacity = {
    "Cincinnati_hub": 95650,
    "Alliance_hub": 44350
}

focus_capacity = {
    "Leipzig_focus": 85000,
    "Hyderabad_focus": 19000,
    "San_Bernadino_focus": 36000
}

demands = [6500, 640, 180, 9100, 570, 19000, 14800, 90, 185, 800, 1700, 170, 2800, 3700, 30, 6700, 190, 175, 38, 2400, 7200, 100, 1200, 1100, 
           1900, 240, 1500, 540, 3400, 185, 1600, 3000, 500, 16, 63, 5100, 172, 200, 173, 300, 290, 550, 1300, 1700, 975, 1200, 480, 100, 450,
           11200, 900, 290, 150, 1200, 420, 1000, 1100, 650, 975, 3300, 3300, 1100, 600, 2000, 260]
demand = {fulfillment_centers[i]: demands[i] for i in range(len(demands))}


In [26]:
total_focus_capacity=sum(focus_capacity.values())
total_hub_capacity=sum(capacity.values())
total_network_capacity=total_focus_capacity + total_hub_capacity
print(f'Total Capacity (Focus Centers & Hubs): {total_network_capacity}')

total_demand=sum(demands)
print(f'Total Demand: {total_demand}')
print(f'Excess Capacity: {total_network_capacity-(total_demand)}')

Total Capacity (Focus Centers & Hubs): 280000
Total Demand: 133747
Excess Capacity: 146253


In [28]:
Xij = { (hub, focus_center): LpVariable(f"Xij_{hub}_{focus_center}", 0, None, LpContinuous) 
        for hub, focus_center in cost_dict.keys() if hub in hubs and focus_center in focus_centers }

Yik = { (hub, fulfillment_center): LpVariable(f"Yik_{hub}_{fulfillment_center}", 0, None, LpContinuous) 
        for hub, fulfillment_center in cost_dict.keys() if hub in hubs and fulfillment_center in fulfillment_centers }

Zjk = { (focus_center, fulfillment_center): LpVariable(f"Zjk_{focus_center}_{fulfillment_center}", 0, None, LpContinuous) 
        for focus_center, fulfillment_center in cost_dict.keys() if focus_center in focus_centers and fulfillment_center in fulfillment_centers }

Define Constraints

In [29]:
# Hub capacity constraint
for hub in hubs:
    prob += (
        lpSum(Xij[hub, focus_center] for focus_center in focus_centers if (hub, focus_center) in Xij) +
        lpSum(Yik[hub, fulfillment_center] for fulfillment_center in fulfillment_centers if (hub, fulfillment_center) in Yik)
        <= capacity[hub]
    ), f"Hub_Capacity_{hub}_Max_{capacity[hub]}"

In [30]:
# Focus center capacity constraint
for focus_center in focus_centers:
    prob += (
        lpSum(Xij[hub, focus_center] for hub in hubs if (hub, focus_center) in Xij)
        <= focus_capacity[focus_center]
    ), f"Focus_Capacity_{focus_center}_Max_{focus_capacity[focus_center]}"

In [31]:
# Quantity Out of a Focus Center
for focus_center in focus_centers:
    prob += (
        lpSum(Xij[hub, focus_center] for hub in hubs if (hub, focus_center) in Xij) ==
        lpSum(Zjk[focus_center, fulfillment_center] for fulfillment_center in fulfillment_centers if (focus_center, fulfillment_center) in Zjk)
    ), f"Flow_Balance_{focus_center}"

In [32]:
# Fulfillment center demand constraint
for fulfillment_center in fulfillment_centers:
    prob += (
        lpSum(Yik[hub, fulfillment_center] for hub in hubs if (hub, fulfillment_center) in Yik) +
        lpSum(Zjk[focus_center, fulfillment_center] for focus_center in focus_centers if (focus_center, fulfillment_center) in Zjk)
        == demand[fulfillment_center]
    ), f"Fulfillment_Demand_{fulfillment_center}"

Define Objective Function

In [33]:
objective = lpSum(
    Xij[hub, focus_center] * cost_dict[(hub, focus_center)]
    for hub, focus_center in Xij.keys()
) + lpSum(
    Yik[hub, fulfillment_center] * cost_dict[(hub, fulfillment_center)]
    for hub, fulfillment_center in Yik.keys()
) + lpSum(
    Zjk[focus_center, fulfillment_center] * cost_dict[(focus_center, fulfillment_center)]
    for focus_center, fulfillment_center in Zjk.keys()
)

prob += objective

Write the Scenario to a .lp file

In [34]:
#prob.writeLP('AmazonLogistics.lp')

Solve

In [35]:
prob.solve(PULP_CBC_CMD(msg=True, options=["primal"]))
print("Status:", LpStatus[prob.status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/d3/xxdfzd3n6lq70gynyy5khm800000gn/T/ed89ab356c2644e984d0b3e18033e4fe-pulp.mps -primal -timeMode elapsed -branch -printingOptions all -solution /var/folders/d3/xxdfzd3n6lq70gynyy5khm800000gn/T/ed89ab356c2644e984d0b3e18033e4fe-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 78 COLUMNS
At line 655 RHS
At line 729 BOUNDS
At line 730 ENDATA
Problem MODEL has 73 rows, 191 columns and 385 elements
Coin0008I MODEL read with 0 errors
Multiple matches for primal - possible completions:
primalP(ivot)
primalS(implex)
primalT(olerance)
primalW(eight)
Option for timeMode changed from cpu to elapsed
Presolve 47 (-26) rows, 144 (-47) columns and 290 (-95) elements
Perturbing problem by 0.001% of 1.6 - largest nonzero change 9.0370431e-05 ( 0.018965159%) - largest zero change 4.412758e-05
0  Ob

<font size = '5'> B: Analyze Output </font>

<font size='4'> B1: Satisfying Optimization Problem Cosntraints: </font>

<b>1. Main Hub Capacity:</b> The two main cargo centers have a limited size. The total quantity of goods each main hub ships out to focus cities (Xij) and fulfillment centers (Yik) can not exceed their storage capacity. The CVG hub’s capacity is 95,650 monthly tons. The AFW hub’s capacity is 44,350 monthly tons.

In [36]:
#Cincinatti
cincinnati_to_focus = 0
cincinnati_to_fulfillment = 0

for focus_center in focus_centers:
    if ('Cincinnati_hub', focus_center) in Xij:
        cincinnati_to_focus += Xij[('Cincinnati_hub', focus_center)].varValue

for fulfillment_center in fulfillment_centers:
    if ('Cincinnati_hub', fulfillment_center) in Yik:
        cincinnati_to_fulfillment += Yik[('Cincinnati_hub', fulfillment_center)].varValue

print(f"Cincinnati sends a total of: {cincinnati_to_focus} to focus centers.")
print(f"Cincinnati sends a total of: {cincinnati_to_fulfillment} to fulfillment centers.")
print(f"Cincinnati sends a total of: {cincinnati_to_focus + cincinnati_to_fulfillment}")
print(f"Cincinnati's Remaining Capacity: {95650 - (cincinnati_to_focus + cincinnati_to_fulfillment)}")

Cincinnati sends a total of: 43470.0 to focus centers.
Cincinnati sends a total of: 52180.0 to fulfillment centers.
Cincinnati sends a total of: 95650.0
Cincinnati's Remaining Capacity: 0.0


In [37]:
#Alliance
alliance_to_focus = 0
alliance_to_fulfillment = 0

for focus_center in focus_centers:
    if ('Alliance_hub', focus_center) in Xij:
        alliance_to_focus += Xij[('Alliance_hub', focus_center)].varValue

for fulfillment_center in fulfillment_centers:
    if ('Alliance_hub', fulfillment_center) in Yik:
        alliance_to_fulfillment += Yik[('Alliance_hub', fulfillment_center)].varValue

print(f"Alliance sends a total of: {alliance_to_focus} to focus centers.")
print(f"Alliance sends a total of: {alliance_to_fulfillment} to fulfillment centers.")
print(f"Alliance sends a total of: {alliance_to_focus + alliance_to_fulfillment}")
print(f"Alliance's Remaining Capacity: {44350 - (alliance_to_focus + alliance_to_fulfillment)}")

Alliance sends a total of: 0.0 to focus centers.
Alliance sends a total of: 38097.0 to fulfillment centers.
Alliance sends a total of: 38097.0
Alliance's Remaining Capacity: 6253.0


<b>2. Quantity Into a Focus Center:</b> Each focus center has a limited size. The total quantity each focus center receives (Xij) can not exceed its storage capacity. The quantity each focus center receives is the sum of goods from main hub 1 and main hub 2. Leipzig focus center’s capacity is 85,000 monthly tons. Hyderabad focus center’s capacity is 19,000 monthly tons. San Bernardino focus center’s capacity is 36,000 monthly tons.


In [38]:
focus_center_capacity = {
    "Leipzig_focus": 85000,
    "San_Bernadino_focus": 36000,
    "Hyderabad_focus": 19000,
}

quantities_into_focus = {focus_center: 0 for focus_center in focus_centers}

for hub in hubs:
    for focus_center in focus_centers:
        if (hub, focus_center) in Xij:
            quantities_into_focus[focus_center] += Xij[(hub, focus_center)].varValue

for focus_center in focus_centers:
    print(f"Total quantity into {focus_center}: {quantities_into_focus[focus_center]}")

Total quantity into Leipzig_focus: 43470.0
Total quantity into Hyderabad_focus: 0
Total quantity into San_Bernadino_focus: 0.0


<b>3. Quantity Out of a Focus Center:</b> Each focus city center receives a certain amount of goods from the main hubs (Xij). The total quantity a focus city center sends out to fulfillment centers (Zjk) is the same quantity they initially received. Therefore, excess goods are not accumulated, and received goods are not lost.


In [39]:
quantities_out_of_focus = {focus_center: 0 for focus_center in focus_centers}

for focus_center in focus_centers:
    for fulfillment_center in fulfillment_centers:
        if (focus_center, fulfillment_center) in Zjk:
            quantities_out_of_focus[focus_center] += Zjk[(focus_center, fulfillment_center)].varValue

for focus_center in focus_centers:
    print(f"{focus_center} received a total of: {quantities_into_focus[focus_center]}")
    print(f"{focus_center} sent a total of: {quantities_out_of_focus[focus_center]}")
    print("-" * 30)

Leipzig_focus received a total of: 43470.0
Leipzig_focus sent a total of: 43470.0
------------------------------
Hyderabad_focus received a total of: 0
Hyderabad_focus sent a total of: 0.0
------------------------------
San_Bernadino_focus received a total of: 0.0
San_Bernadino_focus sent a total of: 0.0
------------------------------


<b>4. Fulfillment Center Demand:</b> Each fulfillment center has a demand for goods based on customer needs. Fulfillment centers in cities with more customers have a higher demand for goods. The goods each fulfillment center receives from the main hubs (Yik) and focus city centers (Zjk) must meet the fulfillment center's demand.


In [48]:
quantities_received_by_fulfillment = {fulfillment_center: 0 for fulfillment_center in fulfillment_centers}

for hub in hubs:
    for fulfillment_center in fulfillment_centers:
        if (hub, fulfillment_center) in Yik:
            quantities_received_by_fulfillment[fulfillment_center] += Yik[(hub, fulfillment_center)].varValue

for focus_center in focus_centers:
    for fulfillment_center in fulfillment_centers:
        if (focus_center, fulfillment_center) in Zjk:
            quantities_received_by_fulfillment[fulfillment_center] += Zjk[(focus_center, fulfillment_center)].varValue

unmet_demand_count = 0

for i, fulfillment_center in enumerate(fulfillment_centers):
    received = quantities_received_by_fulfillment[fulfillment_center]
    demand = demands[i]
    print(f"{fulfillment_center} received a total of: {received}")
    print(f"{fulfillment_center} demand is: {demand}")
    print("-" * 30)
    
    if received < demand:
        unmet_demand_count += 1

print(f"Number of fulfillment centers receiving goods: {len(fulfillment_centers)}")
print(f"Number of fulfillment centers where demand was not met: {unmet_demand_count}")


Paris received a total of: 6500.0
Paris demand is: 6500
------------------------------
Cologne received a total of: 640.0
Cologne demand is: 640
------------------------------
Hanover received a total of: 180.0
Hanover demand is: 180
------------------------------
Bengaluru received a total of: 9100.0
Bengaluru demand is: 9100
------------------------------
Coimbatore received a total of: 570.0
Coimbatore demand is: 570
------------------------------
Delhi received a total of: 19000.0
Delhi demand is: 19000
------------------------------
Mumbai received a total of: 14800.0
Mumbai demand is: 14800
------------------------------
Cagliari received a total of: 90.0
Cagliari demand is: 90
------------------------------
Catania received a total of: 185.0
Catania demand is: 185
------------------------------
Milan received a total of: 800.0
Milan demand is: 800
------------------------------
Rome received a total of: 1700.0
Rome demand is: 1700
------------------------------
Katowice received

<font size = '4'> B2: Decision Variables, Constraints, and Objective Function </font>

<font size = '3'> Decision Variables: </font>

- Values of Each Decision Variable In the Optimal Solution
- Count Non-Zero Decision Variables

In [41]:
#All Decision Varibale Values
for v in prob.variables():
    print(v.name, "=", v.varValue)

Xij_Alliance_hub_San_Bernadino_focus = 0.0
Xij_Cincinnati_hub_Leipzig_focus = 43470.0
Xij_Cincinnati_hub_San_Bernadino_focus = 0.0
Yik_Alliance_hub_Albuquerque = 450.0
Yik_Alliance_hub_Allentown = 420.0
Yik_Alliance_hub_Anchorage = 175.0
Yik_Alliance_hub_Atlanta = 3000.0
Yik_Alliance_hub_Austin = 975.0
Yik_Alliance_hub_Baltimore = 1300.0
Yik_Alliance_hub_Charlotte = 0.0
Yik_Alliance_hub_Chicago = 0.0
Yik_Alliance_hub_Chicago_Rockford = 172.0
Yik_Alliance_hub_Denver = 1500.0
Yik_Alliance_hub_Des_Moines = 300.0
Yik_Alliance_hub_Fairbanks = 38.0
Yik_Alliance_hub_Fort_Wayne = 200.0
Yik_Alliance_hub_Hartford = 540.0
Yik_Alliance_hub_Honolulu = 500.0
Yik_Alliance_hub_Houston = 3300.0
Yik_Alliance_hub_Kahului_Maui = 16.0
Yik_Alliance_hub_Kansas_City = 0.0
Yik_Alliance_hub_Kona = 63.0
Yik_Alliance_hub_Lakeland = 185.0
Yik_Alliance_hub_Los_Angeles = 0.0
Yik_Alliance_hub_Manchester = 100.0
Yik_Alliance_hub_Miami = 0.0
Yik_Alliance_hub_Minneapolis = 1700.0
Yik_Alliance_hub_Mobile = 190.0
Yik_Alli

In [45]:
non_zero_count = sum(1 for v in prob.variables() if v.varValue > 0)
print(f"Number of non-zero decision variables: {non_zero_count}")
total_decision_variables = len(prob.variables())
print(f"Total number of decision variables: {total_decision_variables}")

Number of non-zero decision variables: 67
Total number of decision variables: 191


<font size = '3'> Objective Function </font>

In [43]:
print("Optimal Cost:", value(prob.objective))

Optimal Cost: 199476.25
